In [2]:
import numpy as np
import json
import time
import cv2
from glob import glob

from scipy.spatial.transform import Rotation as R
from scipy.optimize import least_squares

# Eye-in-Hand

## 1. 초기 R, t Load

In [3]:
MAIN_PATH = "/home/ros/llm_robot/data/Calibration/Eye-to-Hand"
with open(f"{MAIN_PATH}/cam2ee.json", "r") as f:
    data = json.load(f)

R_cam2ee = np.array(data["R_cam2ee"])
t_cam2ee = np.array(data["t_cam2ee"]).reshape(3)
q_init = R.from_matrix(R_cam2ee).as_quat()
x0 = np.hstack((q_init, t_cam2ee))

## 2. pose 및 charuco load

In [4]:
sorted_pose_files = sorted(glob(f"{MAIN_PATH}/poses/*.json"))
sorted_charuco_files = sorted(glob(f"{MAIN_PATH}/*.json"))

In [5]:
len(sorted_pose_files), len(sorted_charuco_files[:-1])

(30, 30)

In [ ]:
pose_data = []
for pf, cf in zip(sorted_pose_files, sorted_charuco_files[:-1]):
    with open(pf) as f: pose = json.load(f)
    with open(cf) as f: charuco = json.load(f)
    
    R_base2ee = np.array(pose["R_base2ee"])
    t_base2ee = np.array(pose["t_base2ee"]).reshape(3)
    
    rvec = np.array(charuco["rvec_target2cam"]).reshape(3)
    tvec = np.array(charuco["tvec_target2cam"]).reshape(3)
    R_board2cam = R.from_rotvec(rvec).as_matrix()
    t_board2cam = tvec
    
    pose_data.append({
        "R_base2ee": R_base2ee,
        "t_base2ee": t_base2ee,
        "R_board2cam": R_board2cam,
        "t_board2cam": t_board2cam
    })

## 3. 최적화 함수 

In [7]:
def residuals(x, data):
    q = x[:4]
    t = x[4:]
    R_cam2ee = R.from_quat(q).as_matrix()
    t_cam2ee = t
    res = []
    for d in data:
        # 예측: T_base2board = T_cam2ee × T_base2ee × T_ee2board
        R_pred = R_cam2ee @ d["R_base2ee"] @ d["R_board2cam"]
        t_pred = R_cam2ee @ d["R_base2ee"] @ d["t_board2cam"] + R_cam2ee @ d["t_base2ee"] + t_cam2ee
        
        R_err = R.from_matrix(R_pred).as_rotvec()
        t_err = t_pred  # 실제 board가 base 기준 원점에 있을 필요 없음

        res.extend(R_err)
        res.extend(t_err)
    
    res.append(np.linalg.norm(q) - 1)  # 단위 quaternion 제약
    return np.array(res)

In [9]:
result = least_squares(residuals, x0, args=(pose_data,), method='lm')
result

     message: `ftol` termination condition is satisfied.
     success: True
      status: 2
         fun: [ 2.754e-02 -3.353e-01 ...  3.159e-01  5.270e-08]
           x: [ 6.955e-01  5.667e-01 -2.335e-01 -3.749e-01  1.104e-01
               -1.412e-01  1.630e-01]
        cost: 17.523990818727206
         jac: [[-9.354e-01  6.917e-01 ...  0.000e+00  0.000e+00]
               [-4.769e-01 -7.345e-01 ...  0.000e+00  0.000e+00]
               ...
               [-1.799e-01 -4.107e-02 ...  0.000e+00  1.000e+00]
               [ 6.955e-01  5.667e-01 ...  0.000e+00  0.000e+00]]
        grad: [ 2.136e-04 -2.105e-04 -2.530e-05  9.358e-05 -1.689e-08
               -1.059e-09  1.488e-08]
  optimality: 0.00021357146433181108
 active_mask: [0 0 0 0 0 0 0]
        nfev: 65
        njev: None

## 결과 저장

In [10]:
# === 결과 추출 ===
q_opt = result.x[:4] / np.linalg.norm(result.x[:4])
t_opt = result.x[4:]
R_opt = R.from_quat(q_opt).as_matrix()

# === 저장 (Eye-to-Hand 포맷)
result_dict = {
    "R_cam2ee": R_opt.tolist(),
    "t_cam2ee": (t_opt / 1000).tolist()  # 단위 맞춰서 저장 (m 기준)
}
with open("optimized_cam2ee_LM.json", "w") as f:
    json.dump(result_dict, f, indent=2)

print("✅ 저장 완료: optimized_cam2ee_LM.json")

✅ 저장 완료: optimized_cam2ee_LM.json


# Eye-to-Hand

In [3]:
MAIN_PATH = "/home/ros/llm_robot/data/Calibration/Eye-to-Hand11"

with open(f"{MAIN_PATH}/cam2base_table_normal_fix.json", "r") as f:
    data = json.load(f)

R_cam2base_init = np.array(data["R_cam2base"]).reshape(3, 3)
t_cam2base_init = np.array(data["t_cam2base"]).reshape(3, 1)

In [5]:
sorted_pose_files = sorted(glob(f"{MAIN_PATH}/poses/*.json"))
sorted_charuco_files = sorted(glob(f"{MAIN_PATH}/*_charuco.json"))
len(sorted_pose_files), len(sorted_charuco_files)

(25, 25)

In [6]:
pose_data = []
for pf, cf in zip(sorted_pose_files, sorted_charuco_files):
    with open(pf) as f: pose = json.load(f)
    with open(cf) as f: charuco = json.load(f)
    
    R_base2ee = np.array(pose["R_base2ee"])
    t_base2ee = np.array(pose["t_base2ee"]).reshape(3)
    
    rvec = np.array(charuco["rvec_target2cam"]).reshape(3)
    tvec = np.array(charuco["tvec_target2cam"]).reshape(3, 1)
    R_board2cam = R.from_rotvec(rvec).as_matrix()
    t_board2cam = tvec
    
    pose_data.append({
        "R_base2ee": R_base2ee,
        "t_base2ee": t_base2ee,
        "R_board2cam": R_board2cam,
        "t_board2cam": t_board2cam
    })

## 초기값 추정

In [7]:
def residuals(x, pose_data):
    # x: [rvec(3), tvec(3)]
    rvec_cam2base = x[:3]
    tvec_cam2base = x[3:]
    
    R_cam2base, _ = cv2.Rodrigues(rvec_cam2base)
    t_cam2base = tvec_cam2base.reshape(3, 1)
    
    T_cam2base = np.eye(4)
    T_cam2base[:3, :3] = R_cam2base
    T_cam2base[:3, 3] = t_cam2base.flatten()

    all_residuals = []
    for data in pose_data:
        # Construct the known transformation matrices for the current pose
        R_base2ee = data["R_base2ee"]
        t_base2ee = data["t_base2ee"].reshape(3, 1)
        T_base2ee = np.eye(4)
        T_base2ee[:3, :3] = R_base2ee
        T_base2ee[:3, 3] = t_base2ee.flatten()

        R_board2cam = data["R_board2cam"]
        t_board2cam = data["t_board2cam"].reshape(3, 1)
        T_board2cam = np.eye(4)
        T_board2cam[:3, :3] = R_board2cam
        T_board2cam[:3, 3] = t_board2cam.flatten()

        # Calculate the left and right sides of the core hand-eye equation
        LHS = T_base2ee @ T_cam2base
        RHS = T_cam2base @ T_board2cam

        # Compute the error between the two sides
        # The error is the transformation required to go from RHS to LHS.
        error_matrix = np.linalg.inv(RHS) @ LHS
        
        # Convert the error matrix into a rotation vector and translation vector
        # These vectors form the residuals to be minimized
        error_rvec, _ = cv2.Rodrigues(error_matrix[:3, :3])
        error_tvec = error_matrix[:3, 3]

        # Add the rotation and translation errors to the list of residuals
        all_residuals.extend(error_rvec.flatten())
        all_residuals.extend(error_tvec.flatten())

    return np.array(all_residuals)

In [10]:
rvec_init, _ = cv2.Rodrigues(R_cam2base_init)
x0 = np.concatenate((rvec_init.flatten(), t_cam2base_init.flatten()))

# Run the optimization
res = least_squares(residuals, x0, args=(pose_data,), method='lm')

# Extract and save the optimal result
r_opt = res.x[:3]
t_opt = res.x[3:].reshape(3, 1)
R_opt, _ = cv2.Rodrigues(r_opt)

output = {
    "R_cam2base": R_opt.tolist(),
    "t_cam2base": t_opt.flatten().tolist()
}

with open(f"{MAIN_PATH}/cam2base_LM.json", "w") as f:
    json.dump(output, f, indent=4)

print("Optimized cam2base saved.")

Optimized cam2base saved.


In [11]:
output

{'R_cam2base': [[0.014074200871944553, 0.611250353865447, -0.791312151915527],
  [0.17873624888877834, 0.7771063002381774, 0.6034560062368264],
  [0.9837963560630753, -0.1499293267826468, -0.0983154452134426]],
 't_cam2base': [-0.07404576929589952,
  0.02806981208166249,
  0.06507380795569544]}